In [1]:
exec(open("./funcs/tool_funcs.py").read())



In [2]:
def dijkstra_show_path(adj_matrix, start, end):
 
    n = len(adj_matrix)
    distances = [float('inf')] * n
    previous_nodes = [-1] * n
    distances[start] = 0
    priority_queue = [(0, start)]  # (distance, node)

    while priority_queue:
        current_distance, current_node = heapq.heappop(priority_queue)

        # Stop if we reached the destination node
        if current_node == end:
            break

        # If the distance is no longer optimal, skip
        if current_distance > distances[current_node]:
            continue

        # Explore neighbors
        for neighbor, weight in enumerate(adj_matrix[current_node]):
            if weight > 0:  # Only consider edges with weight > 0
                distance = current_distance + weight

                # Update distance if it's better
                if distance < distances[neighbor]:
                    distances[neighbor] = distance
                    previous_nodes[neighbor] = current_node
                    heapq.heappush(priority_queue, (distance, neighbor))

    # Reconstruct the path from end to start
    path = []
    current = end
    while current != -1:
        path.append(current)
        current = previous_nodes[current]
    path.reverse()

    # If the start node isn't reachable, return an empty path
    return path if path[0] == start else []

In [3]:

def dijkstra_cal_previous_nodes(adj_matrix, start):
 
    n = len(adj_matrix)
    distances = [float('inf')] * n
    previous_nodes = [-1] * n
    distances[start] = 0
    priority_queue = [(0, start)]  

    while priority_queue:
        current_distance, current_node = heapq.heappop(priority_queue)

        if current_distance > distances[current_node]:
            continue
 
        for neighbor, weight in enumerate(adj_matrix[current_node]):
            if weight > 0:  
                distance = current_distance + weight
 
                if distance < distances[neighbor]:
                    distances[neighbor] = distance
                    previous_nodes[neighbor] = current_node
                    heapq.heappush(priority_queue, (distance, neighbor))
 
    return previous_nodes



In [4]:
def dijkstra_show_path_from_previous_nodes(previous_nodes, start, end): 
    path = []
    current = end
    while current != -1:
        path.append(current)
        current = previous_nodes[current]
    path.reverse()
    return path if path[0] == start else []

In [5]:
def translate_path(path, candidate_node_list):
    new_path = [candidate_node_list[i] for i in path]
    return new_path




In [6]:
def output_small_matrix(X_distance_matrix, candidate_node_list):
    small_matrix = np.zeros((len(candidate_node_list), len(candidate_node_list)))
    for i, ii in enumerate(candidate_node_list):
        for j, jj in enumerate(candidate_node_list):
            small_matrix[i,j] = X_distance_matrix[ii, jj]
    return small_matrix




In [7]:
def cal_candidate_node_list(i, j, shortest_path_matrix, X_distance_matrix):
    
    N = len(X_distance_matrix)
    
    remaining_list = [k for k in np.arange(N) if k != i and k != j]
    candidate_node_list = []
    candidate_node_list.append(i)
    for t in remaining_list:
        if shortest_path_matrix[i,j] < shortest_path_matrix[i,t] + shortest_path_matrix[t,j]:
            pass
        else:
            candidate_node_list.append(t)
    candidate_node_list.append(j)
 
    return candidate_node_list
        



In [8]:

def cal_path_warm_start(i, j, shortest_path_matrix, X_distance_matrix):
    candidate_node_list = cal_candidate_node_list(i, j, shortest_path_matrix, X_distance_matrix)
    print("candidate_node_list:")
    print(candidate_node_list)
   
    small_matrix = output_small_matrix(X_distance_matrix, candidate_node_list)
    path = dijkstra_show_path(small_matrix, 0, len(candidate_node_list) - 1)
    path = translate_path(path, candidate_node_list)
    
    return path


In [9]:
# APPD stands for all points path distance (APPD).

def check_shortest_path(i,j, X_distance_matrix, APPD_matrix, path):
    temp1= path[:-1]
    temp2= path[1:]
    llll = [X_distance_matrix[temp1[i],temp2[i]] for i in range(len(temp1))]
    ooo = np.sum(llll)
    return ooo == APPD_matrix[i,j]

In [10]:
def check_if_temp_path_already_in(all_paths_list, temp_path):
    aa = set(temp_path)
    for bb in all_paths_list:
        bb = set(bb)
        if aa == bb:
            return True
    return False

def cal_all_paths_warm_start(i, j, shortest_path_matrix, X_distance_matrix):
    candidate_node_list = cal_candidate_node_list(i, j, shortest_path_matrix, X_distance_matrix)
 
    small_matrix = output_small_matrix(X_distance_matrix, candidate_node_list)
    path = dijkstra_show_path(small_matrix, 0, len(candidate_node_list) - 1)

    all_paths_list = []
    all_paths_list.append(path)

    K = len(candidate_node_list)
    # temp_list = [q for q in range(K) if q not in path]
    
    temp_list = [q for q in range(1, K - 1)]
    
    previous_nodes = dijkstra_cal_previous_nodes(small_matrix, 0)

    for m in temp_list:
        # temp_path1 = dijkstra_show_path(small_matrix, 0, m)
        # There is no need to call dijkstra_show_path for each node m. 
        temp_path1 = dijkstra_show_path_from_previous_nodes(previous_nodes, 0, m)
        
        
        temp_path2 = dijkstra_show_path(small_matrix, m, K - 1)
        temp_path =  temp_path1[:-1] + temp_path2
        if not check_if_temp_path_already_in(all_paths_list, temp_path):
            all_paths_list.append(temp_path)

    all_paths_list = [translate_path(ppp, candidate_node_list) for ppp in all_paths_list]    

    return all_paths_list

In [11]:
# N = 1000

# X_distance_matrix = create_distance_matrix(N)



In [12]:


X_distance_matrix = np.loadtxt("./data/X_100_distance_matrix.csv", delimiter=",")

shortest_path_matrix = floyd_warshall(X_distance_matrix)
 

In [13]:

N = len(X_distance_matrix)


In [14]:
i = 71
j = 93
 
path1 = dijkstra_show_path(X_distance_matrix, i, j)
    
path2 = cal_path_warm_start(i, j, shortest_path_matrix, X_distance_matrix)   
print(path1)
print(path2)


    

candidate_node_list:
[71, 3, 7, 13, 15, 17, 20, 34, 61, 73, 85, 95, 93]
[71, 20, 13, 95, 17, 7, 61, 93]
[71, 20, 13, 95, 17, 7, 61, 93]


In [15]:
check_shortest_path(i,j, X_distance_matrix, shortest_path_matrix, path2)



True

In [16]:
all_paths_list = cal_all_paths_warm_start(i, j, shortest_path_matrix, X_distance_matrix)


In [17]:
for path_i in all_paths_list:
    aaa = check_shortest_path(i, j, X_distance_matrix, shortest_path_matrix, path_i)
    print(path_i, aaa)



[71, 20, 13, 95, 17, 7, 61, 93] True
[71, 3, 85, 15, 34, 73, 7, 61, 93] True


In [18]:
# We can maintain a key_node_list for each pair of nodes, 
# e.g., for the shortest path from node 71 to 93, nodes 7 and 61 are essential, 
# so the key_node_list is [7, 61]. 
# When needing to remove a node, we can just check each pair of nodes' key_node_list 
# to decide if the shortest path is affected.
